# Get ASX Historical Prices from Yahoo using Selnium Webdriver

## Load list of funds in ASX Funds 
1. Download all ASX listed companies from https://www2.asx.com.au/markets/trade-our-cash-market/directory
2. Filter top companies by market cap in each sector that are listed before 01 Jan 2019
3. Save the file as Data\\ASXList.csv
4. Load Data\\ASXList.csv to be processed

In [1]:
#Import all the required librararies
from bs4 import BeautifulSoup
from lxml import html
from selenium import webdriver
import time
import pandas as pd
from selenium.webdriver.edge.service import Service
import json
import os.path
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import numpy as np

In [2]:
#Check if input file Data\\ASX_200.csv exists.
#Set boolExit flag to True if the file does not exit
#The rest of openrations will not run if the file is not present
boolExit = False
if not os.path.isfile(os.path.abspath('Data\\ASXList.csv')):
    print("Data\\ASXList.csv File does not exist. Quiting")
    boolExit = True
else:
    asxDf = pd.read_csv('Data\\ASXList.csv')
#asxDf

## Load the data from output file if already present
Check if output file is already present. 
Load the data in a dataframe if file is present.
This helps in rerun sceanario in case of error 

In [3]:
#Check Data\\historyList.csv exist
#Get list of companies already downloaded
#This will help if the process needs to be rerun in case an error
if boolExit == False:
    if os.path.isfile(os.path.abspath('Data\\historyPriceData.csv')):
        try:
            historyDf = pd.read_csv('Data\\historyPriceData.csv')
            companies = list(historyDf['code'].unique())
        except:
            historyDf = None
            companies =  []
    else:
        historyDf = None
        companies =  []
    if len(companies) >=len(asxDf):
        print('Process Complete')
        boolExit = True
#companies

In [4]:
asxDf['Check'] = asxDf.apply(lambda x: x['code']+ '.AX' if x['code']+ '.AX' not in companies else np.nan, axis=1)
asxDf.dropna(subset=['Check'],inplace=True)

In [5]:
asxDf

,code,Company name,Listing date,GICs industry group,Market Cap,Check
0,BHP,BHP GROUP LIMITED,1885-08-13,Materials,262127094778,BHP.AX
1,CBA,COMMONWEALTH BANK OF AUSTRALIA.,1991-09-12,Banks,181816025300,CBA.AX
2,CSL,CSL LIMITED,1994-06-08,Pharmaceuticals Biotechnology & Life Sciences,126669479707,CSL.AX
3,NAB,NATIONAL AUSTRALIA BANK LIMITED,1962-01-31,Banks,106429144491,NAB.AX
4,WBC,WESTPAC BANKING CORPORATION,1970-07-18,Banks,85042391687,WBC.AX
...,...,...,...,...,...,...
195,CMM,CAPRICORN METALS LTD,2008-07-07,Materials,1686975166,CMM.AX
196,ARF,ARENA REIT.,2013-06-13,Real Estate,1678568166,ARF.AX
197,IPH,IPH LIMITED,2014-11-19,Commercial & Professional Services,1673967125,IPH.AX
198,GUD,G.U.D. HOLDINGS LIMITED,1962-06-30,Automobiles & Components,1665375307,GUD.AX


## Download Historical prices from Yahoo finance

1. Use Selenium Webdriver to open Microsoft Edge
2. Navigate to Historical prices for each ticker
3. Save the price data in Data\\historyList.csv

In [6]:
#Set Selenium Webdriver object
#This will open new Microsoft Edge instance
if boolExit == False:
    #op = webdriver.EdgeOptions()
    s=Service('C:\\Users\\User\\edgedriver_win64\\msedgedriver.exe')
    #driver = webdriver.Edge(service=s, options=op) 
    
    edge_options = webdriver.EdgeOptions()
    edge_options.add_argument("--disable-extensions")
    edge_options.add_argument("--headless")
    driver = webdriver.Edge(service=s, options=edge_options) 
    driver = webdriver.Edge()
    driver.maximize_window()

In [7]:
#Define the function to be called to navigate to Historical data for a defined period
def web_scrape(url):
        # Start the Driver
        executor_url = driver.command_executor._url
        session_id = driver.session_id
        #url = f'https://finance.yahoo.com/quote/{symbol}/history?period1=1449187200&period2=1607040000&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true'

        driver.get(url)
        time.sleep(25)
        driver.switch_to.default_content()

        
        for i in range(0,13):
            driver.execute_script("window.scrollBy(0,5000)")
            time.sleep(5)
            driver.switch_to.default_content()
            # Fetch the webpage and store in a variable.

        # Web page fetched from driver is parsed using Beautiful Soup.
        webpage = driver.page_source
        HTMLPage = BeautifulSoup(driver.page_source, 'html.parser')
        driver.switch_to.default_content()
        return(HTMLPage)

#Tables = HTMLPage.find_all('table', class_='W(100%) M(0)')

# List of all the rows is store in a variable 'Rows'.
#Rows = Tables[0].find_all('tr', class_='BdT Bdc($seperatorColor) Ta(end) Fz(s) Whs(nw)')

#print(Rows)

# Convertinh data received to String
#data_table = html.fromstring(driver.page_source)
#return (Tables, data_table)

In [8]:
#Define output list to save the result
if boolExit == False:
    historyList = []

In [9]:
#Loop through list of ASB 200 companies
#Download the Historical data if the data is not already downloaded
#Conitnue through error to process as many codes as possible
boolError = False
if boolExit == False:
    for asxIndex in asxDf.index:
        #if asxDf.loc[asxIndex]['Code'] + '.AX' not in (companies): 
        symbol = asxDf.loc[asxIndex]['Check']
        #historyUrl = f"https://au.finance.yahoo.com/quote/{symbol}/history?period1=1554076800&period2=1648684800&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true"
        historyUrl = "https://au.finance.yahoo.com/quote/" \
                    + symbol + "/history?period1=1546300800&period2=1648684800" \
                    + "&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true"  

        try:
            historyHtml = web_scrape(historyUrl)
            #print(historyHtml)
            columnsList = historyHtml.find("tr", {"class":"C($tertiaryColor) Fz(xs) Ta(end)"})
            #tr class="C($tertiaryColor) Fz(xs) Ta(end)"
            columns =['code']+[x.text for x in columnsList.find_all('span')]

            rows =   historyHtml.find_all("tr", {"class":"BdT Bdc($seperatorColor) Ta(end) Fz(s) Whs(nw)"})
            #tr class="BdT Bdc($seperatorColor) Ta(end) Fz(s) Whs(nw)"
            for row in rows:
                zip_iter = zip(columns,[symbol]+[x.text for x in row.find_all('span')])
                historyList.append(dict(zip_iter))

                #print(historyList)
                #break                   
        except Exception as e:
            print(symbol)
            #print(str(e))
            #boolError = True
            #continue
            #print(historyList)
            break
        #else:
        #    print(asxDf.loc[asxIndex]['Code'] + '.AX')

KeyboardInterrupt: 

In [ ]:
#Save the results in the Data\\historyList.csv
if boolExit == False:
    if historyDf is None:
        historyListDf = pd.DataFrame(historyList)
    else:
        historyListDf =pd.concat([historyDf, pd.DataFrame(historyList)])
    #print(historyListDf)
    historyListDf.to_csv('Data\\historyPriceData.csv',index=False)

In [ ]:
#Close Microsoft Edge instance
if boolExit == False:
    try:
        driver.quit()
    finally:
        pass